# Magister en Ciencia de Datos - UDD
## DBAnalytics (Ciencia de Datos aplicada)
### SPRINT 3: Datos mapeados x algoritmo I
- Implementar función que genere la variable dependiente a predecir (revisar código WSDMChurnLabeller.scala y descripción en bases de concurso Kaggle)
- Construir el “tablón” final que ingresará al algoritmo (particionando por train/test/validación)
- Entrenar el modelo usando el algoritmo Gradient Boosting Trees (en particular usando las implementaciones XGBoost o LightGBM), para un set de parámetros fijo (exploratorio).

In [1]:
import pandas as pd
import numpy as np
import gc

* Lectura de archivos desde repositorio y contatenacion con nuevas versiones

In [2]:
!gsutil ls -a gs://kk_data_udd

gs://kk_data_udd/day_listen.csv#1564861559107461
gs://kk_data_udd/df_test.csv#1565240427109815
gs://kk_data_udd/df_train.csv#1565240406697715
gs://kk_data_udd/members_v3.csv#1563566790239785
gs://kk_data_udd/sample_submission_v2.csv#1563580288727022
gs://kk_data_udd/sample_submission_zero.csv#1563580145138161
gs://kk_data_udd/sub_age_xgb_pred.csv#1565055727433942
gs://kk_data_udd/sub_day_listen.csv#1564861658307683
gs://kk_data_udd/sub_reg_via_xgb_pred.csv#1565055581900599
gs://kk_data_udd/sub_user_satisfaction.cvs#1564861608790636
gs://kk_data_udd/test_sorted_v1.csv#1565242819914404
gs://kk_data_udd/train.csv#1563565831541482
gs://kk_data_udd/train_sorted_v1.csv#1565243444686022
gs://kk_data_udd/train_v2.csv#1563580263806878
gs://kk_data_udd/transactions.csv#1563580088583483
gs://kk_data_udd/transactions_v2.csv#1563580288931202
gs://kk_data_udd/user_label_201702.csv#1563681052454642
gs://kk_data_udd/user_label_201703.csv#1563681061521361
gs://kk_data_udd/user_latent_satisfaction.csv#1

* Leemos archivos de entrenamiento y test version 1.0

In [3]:
df_train = pd.read_csv('gs://kk_data_udd/train_sorted_v1.csv')

In [4]:
df_test = pd.read_csv('gs://kk_data_udd/test_sorted_v1.csv')

In [5]:
df_train[['city',
 'gender',
 'registered_via',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'discount',
 'membership_days']].head()

,city,gender,registered_via,payment_method_id,payment_plan_days,plan_list_price,discount,membership_days
0,18.0,2.0,9.0,38,30,149,0,0
1,10.0,1.0,9.0,39,30,149,0,0
2,11.0,2.0,9.0,39,30,149,0,0
3,13.0,2.0,9.0,40,30,149,0,0
4,3.0,1.0,9.0,38,90,477,0,0


In [6]:
def GetINT(obj):
    return str(obj).split('.')[0] 

In [7]:
def GetINT2(obj):
    return str(obj)

In [8]:
df_train['city'] = df_train['city'].apply(GetINT)
df_cities = pd.get_dummies(df_train[['city']], prefix=["city"])

df_train['gender'] = df_train['gender'].apply(GetINT)
df_gender = pd.get_dummies(df_train[['gender']], prefix=["gender"])

df_train['registered_via'] = df_train['registered_via'].apply(GetINT)
df_registered_via = pd.get_dummies(df_train[['registered_via']], prefix=["registered_via"])

In [9]:
df_train['payment_method_id'] = df_train['payment_method_id'].apply(GetINT2)
df_payment_method_id = pd.get_dummies(df_train[['payment_method_id']], prefix=["pmi"])

df_train['payment_plan_days'] = df_train['payment_plan_days'].apply(GetINT2)
df_payment_plan_days = pd.get_dummies(df_train[['payment_plan_days']], prefix=["ppd"])

df_train['plan_list_price'] = df_train['plan_list_price'].apply(GetINT2)
df_plan_list_price = pd.get_dummies(df_train[['plan_list_price']], prefix=["plp"])

df_train['discount'] = df_train['discount'].apply(GetINT2)
df_discount = pd.get_dummies(df_train[['discount']], prefix=["discount"])

df_train['membership_days'] = df_train['membership_days'].apply(GetINT2)
df_membership_days = pd.get_dummies(df_train[['membership_days']], prefix=["md"])

In [10]:
len(df_train.columns)

66

In [11]:
len(df_cities.columns)

22

In [12]:
df_train=pd.concat([df_train, df_cities],axis=1)

In [13]:
len(df_train.columns)

88

In [14]:
df_train=pd.concat([df_train, df_gender],axis=1)
df_train=pd.concat([df_train, df_registered_via],axis=1)
df_train=pd.concat([df_train, df_payment_method_id],axis=1)
df_train=pd.concat([df_train, df_payment_plan_days],axis=1)
df_train=pd.concat([df_train, df_plan_list_price],axis=1)
df_train=pd.concat([df_train, df_discount],axis=1)
df_train=pd.concat([df_train, df_membership_days],axis=1)

In [15]:
df_train.shape

(1963891, 217)

In [16]:
!gsutil ls -a gs://kk_data_udd

gs://kk_data_udd/day_listen.csv#1564861559107461
gs://kk_data_udd/df_test.csv#1565240427109815
gs://kk_data_udd/df_train.csv#1565240406697715
gs://kk_data_udd/members_v3.csv#1563566790239785
gs://kk_data_udd/sample_submission_v2.csv#1563580288727022
gs://kk_data_udd/sample_submission_zero.csv#1563580145138161
gs://kk_data_udd/sub_age_xgb_pred.csv#1565055727433942
gs://kk_data_udd/sub_day_listen.csv#1564861658307683
gs://kk_data_udd/sub_reg_via_xgb_pred.csv#1565055581900599
gs://kk_data_udd/sub_user_satisfaction.cvs#1564861608790636
gs://kk_data_udd/test_sorted_v1.csv#1565242819914404
gs://kk_data_udd/train.csv#1563565831541482
gs://kk_data_udd/train_sorted_v1.csv#1565243444686022
gs://kk_data_udd/train_v2.csv#1563580263806878
gs://kk_data_udd/transactions.csv#1563580088583483
gs://kk_data_udd/transactions_v2.csv#1563580288931202
gs://kk_data_udd/user_label_201702.csv#1563681052454642
gs://kk_data_udd/user_label_201703.csv#1563681061521361
gs://kk_data_udd/user_latent_satisfaction.csv#1

In [17]:
!gsutil ls -a gs://kk_data_udd/submission/

gs://kk_data_udd/submission/sub_day_listen_bins_pred.csv#1564861912140714
gs://kk_data_udd/submission/sub_orginal_day_listen_pred.csv#1564861906258198
gs://kk_data_udd/submission/sub_reg_via_xgb_pred.csv#1563929337202673


In [18]:
#user_latent_satisfaction
df_user_latent_satisfaction = pd.read_csv('gs://kk_data_udd/user_latent_satisfaction.csv')

In [19]:
df_user_latent_satisfaction.head()

,msno,user_latent_satisfaction
0,+2oK/qWmYvAnfNZsVV5pdsJ9n6d/LZn6CdwiJajGZas=,0.555549
1,+iMj+cgRiFJrmnOOLGwEZu6nzH0KbsMIdOZkpItAXTs=,0.798740
2,10QzA1bL7qK5Oeec6OlbyUEBOipzjY4OHN8gf2xokW4=,0.512925
3,2Dvm0YBqp+gDTOksgoR5ixNOk64C6ozpBzEkhbdkTrE=,0.958960
4,7B2xf5mD+GijSROhxSmKDuybhy+H5rgjfSy9sy6Xt4g=,0.762336


In [20]:
# day_listen
df_day_listen = pd.read_csv('gs://kk_data_udd/day_listen.csv')

In [21]:
df_day_listen.head()

,msno,is_churn,day_listen
0,+MFM2WIg07EADKRI0gm/Vq+3s2jDW838pVt477htI8k=,False,74.0
1,/fBID42G+PGSb8jQ4Lo0p6DI6gX78OIp5/thoFYn68M=,False,27.0
2,/qu3BCDcv1UqIHl2kal9kqrWlpYy4QMW5LolR5fR5/A=,False,126.0
3,/reu7jVe3U4hlk366ZmiR/YkVi6H/C2SSa8J4tSQtsQ=,False,66.0
4,1VIbdCaMTd1AKazZRzQDDrtO+emWOrncVaGRxEG8eiY=,False,130.0


In [22]:
del df_day_listen['is_churn']

In [23]:
df_train = pd.merge(df_train, df_day_listen, on='msno')

In [24]:
df_train = pd.merge(df_train,df_user_latent_satisfaction, on='msno')

In [25]:
df_train.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,discount_120,discount_149,discount_180,discount_20,discount_30,discount_50,md_-1,md_0,day_listen,user_latent_satisfaction
0,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10,38.0,1,9,2005.0,39,30,149,...,0,0,0,0,0,0,0,1,33.0,0.820694
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10,38.0,1,9,2005.0,39,30,149,...,0,0,0,0,0,0,0,1,33.0,0.820694
2,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,1,18,22.0,2,9,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,152.0,0.919394
3,Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,0,18,22.0,2,9,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,152.0,0.919394
4,MkuWz0Nq6/Oq5fKqRddWL7oh2SLUSRe3/g+XmAWqW1Q=,1,11,31.0,2,9,2006.0,38,30,149,...,0,0,0,0,0,0,0,1,4.0,1.000000


In [26]:
df_train.shape

(1667562, 219)

In [27]:
df_train.to_csv("gs://kk_data_udd/train_sorted_v2.csv", index=False)

### Feature Selection

In [18]:
import numpy as np 
import pandas as pd 

import lightgbm as lgb
from sklearn.feature_selection import RFE

In [19]:
df = pd.read_csv('gs://kk_data_udd/train_sorted_v2.csv')

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().values.sum()

In [ ]:
df = df.fillna(0)

In [ ]:
df.columns

In [41]:
cols = [c for c in df.columns if c not in ['is_churn', 'msno']]

In [42]:
train = df[cols]
target = df.is_churn

Importancia de las features y el ajuste del modelo lightGBM

In [43]:
gbm = lgb.LGBMRegressor()
gbm.fit(train, target)
gbm.booster_.feature_importance()

array([ 21,  37,   7,  10,  57,  97,  56,  51,  37,  41,  31,  23,  65,
         6,   0,  38,  57,  47,  12,  14,   0,  26,  26,  18,  31,   0,
        26,  17,  24,  28,   0,  15,  25,  25,  28,   0,  17,  25,  24,
        19,   0,  18,   8,  10,  21,   3,  21,  23,  12,  11,  46,  22,
        18,  14,  35,  31, 137, 326, 102, 305,  27, 118,  49,   0,   0,
         1,   0,   0,   2,   1,   2,   0,   0,   0,   0,   0,   0,   0,
         1,   3,   0,   0,   0,   0,   0,   0,   5,   0,   0,   0,   0,
         6,  32,   0,   0,   0,   0,   0,   3,   0,  11,   0,   0,   0,
         0,   0,   0,   7,   0,   4,   0,   2,   9,   0,   0,   0,   8,
         6,   2,   2,  33,   6,  13,  40,  40,   0,   0,   0,   0,   0,
         0,   0,   0,   3,   0,   3,   0,   5,   0,   0,   0,   0,   0,
        25,   0,   6,   3,   0,   1,   1,   0,   0,   0,   2,   0,   0,
         0,   1,   0,   0,   3,   0,   0,   0,   0,   2,   0,   0,   1,
         0,   0,   0,   0,   0,   6,   0,   1,   0,   1,   0,   

Importancia de cada feature

In [44]:
fea_imp_ = pd.DataFrame({'feature':train.columns, 'feature_importance':gbm.feature_importances_})
fea_imp_.loc[fea_imp_.feature_importance > 0].sort_values(by=['feature_importance'], ascending = False)

,feature,feature_importance
57,m_day,326
59,t_day,305
215,day_listen,173
56,m_month,137
61,l_day,118
58,t_month,102
5,payment_method_id,97
12,is_cancel,65
16,amt_per_day,57
4,registration_init_time,57


Recursive Feature Elimination(RFE)

In [25]:
# Instanciamos el modelo RFE y selecciona 10 features
rfe = RFE(gbm, 10)
rfe = rfe.fit(train, target)

In [26]:
# Resume la selección de los features
print(rfe.support_)

[False False False False False  True False  True False False False False
 False False False False  True  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True  True  True  True
 False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [27]:
# Ranking de los feautres más relevantes
fea_rank_ = pd.DataFrame({'cols':train.columns, 'fea_rank':rfe.ranking_})
fea_rank_.loc[fea_rank_.fea_rank > 0].sort_values(by=['fea_rank'], ascending = True)

,cols,fea_rank
56,m_month,1
57,m_day,1
58,t_month,1
59,t_day,1
5,payment_method_id,1
7,plan_list_price,1
61,l_day,1
215,day_listen,1
17,msno_nro,1
16,amt_per_day,1


In [32]:
fea_rank_.loc[fea_rank_.fea_rank > 0].sort_values(by=['fea_rank'], ascending = True).to_csv("gs://kk_data_udd/fea_rank.csv", index=False)

### Modelamos con xgboost

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import xgboost as xgb
import pandas as pd

# ignore warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [21]:
train = pd.read_csv('gs://kk_data_udd/train_sorted_v2.csv')

In [22]:
test = pd.read_csv('gs://kk_data_udd/test_sorted_v1.csv')

In [23]:
df_day_listen = pd.read_csv('gs://kk_data_udd/day_listen.csv')
del df_day_listen['is_churn']
test = pd.merge(test, df_day_listen, on='msno')

In [24]:
df_user_latent_satisfaction = pd.read_csv('gs://kk_data_udd/user_latent_satisfaction.csv')
test = pd.merge(test,df_user_latent_satisfaction, on='msno')

In [25]:
test.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,...,m_month,m_day,t_month,t_day,l_month,l_day,autorenew_&_not_cancel,notAutorenew_&_cancel,day_listen,user_latent_satisfaction
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,1.0,0.0,NaN,7.0,2015.0,41,30,99,...,4,18,3,18,1.0,15.0,1,1,1.0,0.000000
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,4.0,28.0,1.0,9.0,2005.0,34,30,149,...,4,30,3,31,3.0,31.0,1,1,178.0,0.515119
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,4.0,34.0,1.0,7.0,2014.0,41,30,99,...,4,15,3,15,3.0,28.0,1,1,44.0,0.665889
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,NaN,NaN,NaN,NaN,NaN,41,30,99,...,4,27,3,27,NaN,NaN,1,1,NaN,NaN
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0,1.0,0.0,NaN,13.0,2016.0,30,30,129,...,4,21,3,22,2.0,16.0,1,1,16.0,0.885021


In [26]:
test = test.drop_duplicates('msno')

In [27]:
test = test.fillna(0)

In [28]:
train = train.fillna(0)

In [29]:
top_10 = ['m_month' ,'m_day','t_month','t_day','payment_method_id','plan_list_price','l_day','day_listen','msno_nro','amt_per_day','num_50_std','is_cancel'	,'payment_plan_days','num_98_sum','total_secs_min','autorenew_&_not_cancel','user_latent_satisfaction','registration_init_time','num_25_max']

In [30]:
test = test[top_10]

In [31]:
training_features = train[top_10]
training_target = train['is_churn']
#target = train["Churn"]

In [32]:
xgtrain = xgb.DMatrix(training_features.values, training_target.values)
xgtest = xgb.DMatrix(test.values)

In [33]:
params = { 
   "objective": "binary:logistic",
   "booster": "gbtree",
   "eval_metric": "auc",
   "eta": 0.01, 
   "subsample": 0.75,
   "colsample_bytree": 0.68,
   "max_depth": 7
}

In [34]:
clf = xgb.train(params,\
                xgtrain,\
                verbose_eval=True,\
                maximize=False)

In [35]:
#Make predict
print('Predict...')
test_preds = clf.predict(xgtest, ntree_limit=clf.best_iteration)

Predict...


In [36]:
test_preds

array([0.456817  , 0.4569834 , 0.456817  , ..., 0.456817  , 0.45717365,
       0.456817  ], dtype=float32)